In [12]:
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
SOURCE_FILE = "./data/dane_puste_przebiegi_cleaned.xlsx"
SAMPLE_DATA_FILE = "./data/output/sample_de/sample_dane_puste_przebiegi.xlsx"

DISTANCE_FILE = "./data/output/sample_de/sample_de_cities_distance_matrix.xlsx"
DISTANCE_FILE_SHEET = "Cities_Distances_Matrix"

TIME_FILE = "./data/output/sample_de/sample_de_cities_time_matrix.xlsx"
TIME_FILE_SHEET = "Cities_Time_Matrix"

In [19]:
df = pd.read_excel(SOURCE_FILE)
df

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,26692/2025,129.0,WGM9815L,(DE) Nürnberg - (AT) Wörgl - (DE) Nürnberg,nürnberg,90475,DE,nürnberg,90475,DE,2025-09-08 00:30:00,2025-09-08 00:46:00,2025-09-08 22:00:00,2025-09-08 22:15:00,0.000,553.013,553.013
1,27082/2025,105.0,PZ4S023,(PL) Jarosty - (CZ) Praha 5,jarosty,97-310,PL,praha 5,NaN,CZ,2025-09-08 01:10:00,2025-09-07 23:55:00,2025-09-09 03:15:00,2025-09-09 01:47:27,0.000,540.058,540.058
2,27028/2025,61.0,PZ4R993,(DE) Euskirchen - (FR) amiens,euskirchen,53881,DE,amiens,80013,FR,2025-09-08 05:00:00,2025-09-08 16:32:00,2025-09-08 15:00:00,2025-09-09 11:16:00,0.000,409.626,409.626
3,27026/2025,47.0,WGM8283K,(CZ) Modřice - (CZ) Pelhřimov - (CZ) VRSKMAN,modřice,66442,CZ,vrskman,43111,CZ,2025-09-08 05:30:00,2025-09-08 05:48:28,2025-09-08 12:30:00,2025-09-08 14:04:21,318.425,351.447,669.872
4,27049/2025,105.0,WGM5118H,(PL) Chlastawa - (DE) Berlin,chlastawa,66-210,PL,berlin,13597,DE,2025-09-08 06:00:00,2025-09-09 00:34:00,2025-09-09 06:00:00,2025-09-09 05:17:00,0.789,216.089,216.878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,29273/2025,61.0,WGM9817L,(FR) amiens - (DE) Langenbach,amiens,80013,FR,langenbach,85416,DE,2025-09-27 10:00:00,2025-09-27 13:31:00,2025-09-30 15:30:00,NaT,0.000,913.536,913.536
1639,29398/2025,105.0,WGM5121H,(PL) Osieczna - (CZ) Praha - CERNY MOST,osieczna,64-113,PL,praha - cerny most,19800,CZ,2025-09-27 10:00:00,2025-09-27 20:10:04,2025-09-29 20:45:00,NaT,0.973,372.595,373.568
1640,29444/2025,90.0,PZ911JJ,(PL) OSIECZNA - (PL) Wiskitki,osieczna,64-113,PL,wiskitki,NaN,PL,2025-09-27 12:00:00,2025-09-26 14:09:12,2025-09-27 17:00:00,2025-09-27 20:12:12,0.000,311.320,311.320
1641,29515/2025,61.0,PO2PY72,(PL) OSIECZNA - (PL) Sochaczew,osieczna,64-113,PL,sochaczew,96-500,PL,2025-09-27 14:00:00,NaT,2025-10-01 03:00:00,NaT,0.000,316.000,316.000


# Limiting the scope of data

Warunki:
- okres miedzy planowanym załadunkiem 2025-09-08 & planowanym rozładunkiem 2025-09-14
- trasy skupiajace sie na maistach w niemczech (country code DE)
    - jesli jakaś trasa w tym oresie miała tylko fragment po Niemczech okrawamy ja do tego odcinka

Przykład:

| idx | load_country | unload_country | Uwagi                   |
| --- | ------------ | -------------- | ----------------------- |
| 0   | PL           | CZ             | ❌ (obie ≠ DE) – usuwamy |
| 1   | CZ           | DE             | ✅ zaczyna się blok DE   |
| 2   | DE           | NL             | ✅ (powiązana z DE)      |
| 3   | NL           | BE             | ⚪ zostaje – między DE   |
| 4   | BE           | DE             | ✅ (powiązana z DE)      |
| 5   | DE           | FR             | ✅ (powiązana z DE)      |
| 6   | FR           | IT             | ❌ (obie ≠ DE) – usuwamy |


In [ ]:

df["pickup_planned"] = pd.to_datetime(df["pickup_planned"], errors="coerce")
df["delivery_planned"] = pd.to_datetime(df["delivery_planned"], errors="coerce")

germany_variants = ["de", "deu", "germany", "deutschland", "niemcy", "ger"]
df["load_country"] = df["load_country"].astype(str).str.strip().str.lower()
df["unload_country"] = df["unload_country"].astype(str).str.strip().str.lower()

mask_date = (
    (df["pickup_planned"] >= "2025-09-08") &
    (df["delivery_planned"] <= "2025-09-14")
)
df_period = df.loc[mask_date].copy()


trimmed_groups = []

for tractor, group in df_period.groupby("tractor"):
    group = group.sort_values("pickup_planned").reset_index(drop=True)

    # Trasa związana z Niemcami — jeśli load lub unload w DE
    mask_related_to_de = (
        group["load_country"].isin(germany_variants) |
        group["unload_country"].isin(germany_variants)
    )

    # Jeżeli żadna trasa nie dotyczy Niemiec → pomijamy
    if not mask_related_to_de.any():
        continue

    # Indeksy pierwszej i ostatniej trasy związanej z DE
    first_idx = mask_related_to_de.idxmax()
    last_idx = mask_related_to_de[::-1].idxmax()

    # Przycięcie — zachowujemy wszystko między nimi (łącznie)
    trimmed = group.loc[first_idx:last_idx].copy()
    trimmed_groups.append(trimmed)

# --- Połączenie przyciętych grup
df_week2_de = pd.concat(trimmed_groups, ignore_index=True)


df_week2_de = df_week2_de.sort_values(["tractor", "pickup_planned"]).reset_index(drop=True)

print(f"🚛 Ciągników po przycięciu: {df_week2_de['tractor'].nunique()}")
print(f"📦 Tras po przycięciu: {len(df_week2_de)}")


🚛 Ciągników po przycięciu: 128
📦 Tras po przycięciu: 360


In [4]:
df_week2_de

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,27070/2025,61.0,PO2PY63,(DE) Pritzwalk - (HU) Csomor,pritzwalk,16928,de,csömör,2141,hu,2025-09-08 07:00:00,2025-09-08 06:47:13,2025-09-09 07:30:00,2025-09-09 16:25:00,137.687,1180.231,1317.918
1,27356/2025,35.0,PO2PY63,(HU) Csömör - (DE) CRAILSHEIM,csömör,2141,hu,crailsheim,74564,de,2025-09-10 04:00:00,2025-09-10 14:14:18,2025-09-11 14:00:00,2025-09-11 15:25:00,3.267,898.686,901.953
2,27576/2025,61.0,PO2PY63,(DE) CRAILSHEIM - (DE) Landsberg,crailsheim,74564,de,landsberg,06188,de,2025-09-11 08:00:00,2025-09-11 15:25:14,2025-09-12 18:00:00,2025-09-15 08:00:00,0.000,383.061,383.061
3,27165/2025,35.0,PO2PY67,(FR) amiens - (DE) KIEL,amiens,80013,fr,kiel,24109,de,2025-09-08 17:00:00,2025-09-08 18:30:00,2025-09-10 06:00:00,2025-09-10 08:35:00,199.784,899.613,1099.397
4,27393/2025,47.0,PO2PY67,(DE) Hamburg - (PL) OSIECZNA,hamburg,21129,de,osieczna,64-113,pl,2025-09-10 09:00:00,2025-09-10 13:00:00,2025-09-11 18:00:00,2025-09-11 19:28:47,101.892,592.303,694.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,27283/2025,61.0,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-09 17:00:00,2025-09-10 03:06:00,2025-09-10 06:00:00,2025-09-10 12:02:35,0.000,517.388,517.388
356,27479/2025,61.0,WPR1249U,(DE) BERLIN - (PL) Łódź,berlin,12099,de,łódź,94-406,pl,2025-09-10 10:00:00,2025-09-10 12:45:00,2025-09-10 19:00:00,2025-09-10 22:19:00,0.000,519.184,519.184
357,27485/2025,61.0,WPR1249U,(PL) Łódź - (DE) BERLIN,łódź,94-406,pl,berlin,12099,de,2025-09-10 21:00:00,2025-09-10 22:23:00,2025-09-11 10:00:00,2025-09-11 15:12:00,0.000,517.388,517.388
358,27493/2025,61.0,WPR1249U,(DE) BERLIN - (PL) Łódź,berlin,12099,de,łódź,94-406,pl,2025-09-11 10:00:00,2025-09-11 15:28:00,2025-09-11 19:00:00,2025-09-12 00:14:00,0.000,519.184,519.184


In [ ]:
df_week2_de.to_excel(SAMPLE_DATA_FILE, index=False)

## Lista miast w próbce

In [16]:
# Utworzenie listy par (miasto, kraj) z kolumn załadunku i rozładunku
load_pairs = df_week2_de[["load_city", "load_country"]].apply(tuple, axis=1)
unload_pairs = df_week2_de[["unload_city", "unload_country"]].apply(tuple, axis=1)

# Połączenie list i usunięcie duplikatów oraz NaN-ów
city_country_pairs = load_pairs.tolist() + unload_pairs.tolist()
city_country_pairs = [pair for pair in city_country_pairs if pd.notna(pair[0]) and pd.notna(pair[1])]

# Unikalne i posortowane pary
city_country_pairs = sorted(set(city_country_pairs))

print(f"Liczba unikalnych par miasto / kraj: {len(city_country_pairs)}")

city_country_pairs_df = pd.DataFrame(city_country_pairs, columns=["city", "country_code"])


Liczba unikalnych par miasto / kraj: 230


In [6]:
city_country_pairs_df

,city,country_code
0,agnadello,it
1,altfeld,de
2,alzenau,de
3,amiens,fr
4,amt wachsenburg,de
...,...,...
225,zbaszynek,pl
226,zgorzelec,pl
227,zuzenhausen,de
228,zwijndrecht,be


In [11]:
sample_cities_distance_de_matrix = pd.DataFrame(
    data=0,
    index=city_country_pairs_df['city'],
    columns=city_country_pairs_df['city'],
)

# Zapis do Excela
sample_cities_distance_de_matrix.to_excel(DISTANCE_FILE, sheet_name=DISTANCE_FILE_SHEET)
sample_cities_distance_de_matrix.to_excel(TIME_FILE, sheet_name=TIME_FILE_SHEET)

print(f"Macierz została zapisana do pliku: {DISTANCE_FILE} & {TIME_FILE}")

Macierz została zapisana do pliku: ./data/output/sample_de_2/sample_de_cities_distance_matrix.xlsx & ./data/output/sample_de_2/sample_de_cities_time_matrix.xlsx


# Wypełnianie macierzy

In [10]:
output_file = "./data/output/sample_de/sample_cities_distance_de_matrix.xlsx"

sample_cities_distance_de_matrix = pd.read_excel(output_file, index_col=0)

sample_cities_distance_de_matrix.head()


,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
altfeld,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alzenau,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
amiens,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
amt wachsenburg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Wypełnianie z zachowanych uzupełnień

In [11]:
old_sample_file_path = "./data/output/sample_de/sample_cities_distance_de_matrix copy 4.xlsx"

old_sample_cities_distance_de_matrix = pd.read_excel(old_sample_file_path, index_col=0)

old_sample_cities_distance_de_matrix.head()

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00000,689.69838,736.32488,1028.03563,821.21306,807.32656,378.07547,451.18134,966.16325,900.30200,...,658.59669,579.96481,578.05044,657.16956,1506.68350,1285.57825,1088.16225,628.25413,1089.70238,1447.04850
altfeld,689.69838,0.00000,66.21271,667.40900,233.55459,206.15992,403.82131,624.65481,638.71938,278.81241,...,144.03942,160.91139,217.03716,40.03594,1879.14638,720.83656,523.42053,152.45614,514.74544,903.16225
alzenau,736.32488,66.21271,0.00000,632.98106,261.08945,171.73195,466.33969,599.53713,604.29144,221.53011,...,109.61146,223.42977,182.60919,102.55432,1854.02875,738.13944,579.28138,164.60931,480.31750,1004.02994
amiens,1028.03563,667.40900,632.98106,0.00000,763.55469,471.23153,814.44144,559.88975,137.59623,520.23456,...,547.10169,742.15025,599.31831,701.89394,1658.59125,1145.22863,1081.74663,603.24275,346.00463,1411.11913
amt wachsenburg,821.21306,233.55459,261.08945,763.55469,0.00000,309.84056,540.47069,798.58956,817.31006,333.11663,...,308.66388,292.81534,381.66159,181.77248,2053.08125,487.07478,328.21669,329.34228,590.78588,752.96525


In [12]:
aligned_old = old_sample_cities_distance_de_matrix.reindex_like(sample_cities_distance_de_matrix)

# Zaktualizowanie tylko tam, gdzie w old ≠ 0
sample_cities_distance_de_matrix = sample_cities_distance_de_matrix.where(
    aligned_old == 0, aligned_old
)

In [13]:
sample_cities_distance_de_matrix.head()

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00000,689.69838,736.32488,1028.03563,821.21306,807.32656,378.07547,451.18134,966.16325,900.30200,...,658.59669,579.96481,578.05044,657.16956,1506.68350,1285.57825,1088.16225,628.25413,1089.70238,1447.04850
altfeld,689.69838,0.00000,66.21271,667.40900,233.55459,206.15992,403.82131,624.65481,638.71938,278.81241,...,144.03942,160.91139,217.03716,40.03594,1879.14638,720.83656,523.42053,152.45614,514.74544,903.16225
alzenau,736.32488,66.21271,0.00000,632.98106,261.08945,171.73195,466.33969,599.53713,604.29144,221.53011,...,109.61146,223.42977,182.60919,102.55432,1854.02875,738.13944,579.28138,164.60931,480.31750,1004.02994
amiens,1028.03563,667.40900,632.98106,0.00000,763.55469,471.23153,814.44144,559.88975,137.59623,520.23456,...,547.10169,742.15025,599.31831,701.89394,1658.59125,1145.22863,1081.74663,603.24275,346.00463,1411.11913
amt wachsenburg,821.21306,233.55459,261.08945,763.55469,0.00000,309.84056,540.47069,798.58956,817.31006,333.11663,...,308.66388,292.81534,381.66159,181.77248,2053.08125,487.07478,328.21669,329.34228,590.78588,752.96525


### a) matrix_loaded_km file

In [6]:
old_sample_file_path = "./data/old/matrix_loaded_km.xlsx"

In [7]:
old_matrix_loaded_km_df = pd.read_excel(old_sample_file_path, index_col=0)
old_matrix_loaded_km_df.head()

,aalen,abanilla,ablis,abrera,agnadello,alicante,alizay,altfeld,alzenau,alzey,...,öhringen,ühlingen-birkendorf,łódź,říčany,środa śląska gmina,świebodzice,świebodzin,święciechowa,żary,žirovnice
aalen,0.00000,1871.69250,782.61925,1360.63963,528.73881,1870.00663,850.73963,175.59844,237.6895,256.50864,...,112.75442,247.18225,919.36688,432.02303,669.09244,677.28506,704.9435,724.14044,621.93344,546.01844
abanilla,1871.69250,0.00000,1506.10088,523.93488,1551.08988,74.53752,1617.61738,1922.52075,1892.4405,1793.28625,...,1829.45000,1607.90538,2751.16350,2263.81975,2500.88900,2509.08175,2541.2220,2555.93700,2453.73000,2377.81500
ablis,782.61925,1506.10088,0.00000,995.63863,969.82863,1505.00563,148.80500,700.08525,661.5140,558.47463,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000
abrera,1360.63963,523.93488,995.63863,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000
agnadello,528.73881,1551.08988,969.82863,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000


In [8]:
# Dopasowanie indeksów i kolumn
aligned_old = old_matrix_loaded_km_df.reindex_like(sample_cities_distance_de_matrix)

# Zaktualizowanie tylko tam, gdzie w old ≠ 0
sample_cities_distance_de_matrix = sample_cities_distance_de_matrix.where(
    aligned_old == 0, aligned_old
)

In [9]:
sample_cities_distance_de_matrix.head()

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
altfeld,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alzenau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amiens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amt wachsenburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### b) sample_matrix_loaded_km_08_14 file

In [10]:
old_sample_file_path = "./data/old/sample_matrix_loaded_km_08_14.xlsx"

In [11]:
old_sample_matrix_loaded_km_08_14_df = pd.read_excel(old_sample_file_path, index_col=0)
old_sample_matrix_loaded_km_08_14_df.head()

,agnadello,alicante,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,...,zarrentin,zbaszynek,zele,zgorzelec,zoetermeer,zuzenhausen,zwijndrecht,łódź,środa śląska gmina,świebodzice
agnadello,0.00000,1550.45475,689.69838,736.32488,1028.03563,821.21306,807.32656,378.07547,451.18134,966.16325,...,1233.34738,1285.57825,999.89000,1088.16225,1124.48550,628.25413,1089.70238,1447.04850,1217.62950,1225.82200
alicante,1550.45475,0.00000,1921.34025,1891.25988,1712.48325,2095.08913,1855.04213,1658.76338,1355.35775,1577.91375,...,2388.28250,2572.13900,1874.57850,2370.24125,2029.82100,1783.18913,1986.16350,2749.98300,2499.70850,2507.90125
altfeld,689.69838,1921.34025,0.00000,66.21271,667.40900,233.55459,206.15992,403.82131,624.65481,638.71938,...,611.56513,720.83656,505.68422,523.42053,523.31884,152.45614,514.74544,903.16225,652.88775,661.08038
alzenau,736.32488,1891.25988,66.21271,0.00000,632.98106,261.08945,171.73195,466.33969,599.53713,604.29144,...,554.28281,738.13944,471.25628,579.28138,488.89091,164.60931,480.31750,1004.02994,708.74863,716.94119
amiens,1028.03563,1712.48325,667.40900,632.98106,0.00000,763.55469,471.23153,814.44144,559.88975,137.59623,...,885.82300,1145.22863,234.41958,1081.74663,389.66203,603.24275,346.00463,1411.11913,1211.21388,1219.40650


In [12]:
# Dopasowanie indeksów i kolumn
aligned_old = old_sample_matrix_loaded_km_08_14_df.reindex_like(sample_cities_distance_de_matrix)

# Zaktualizowanie tylko tam, gdzie w old ≠ 0
sample_cities_distance_de_matrix = sample_cities_distance_de_matrix.where(
    aligned_old == 0, aligned_old
)

In [13]:
sample_cities_distance_de_matrix.head()

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00000,689.69838,736.32488,1028.03563,821.21306,807.32656,378.07547,451.18134,966.16325,900.30200,...,658.59669,579.96481,578.05044,657.16956,1506.68350,1285.57825,1088.16225,628.25413,1089.70238,1447.04850
altfeld,689.69838,0.00000,66.21271,667.40900,233.55459,206.15992,403.82131,624.65481,638.71938,278.81241,...,144.03942,160.91139,217.03716,40.03594,1879.14638,720.83656,523.42053,152.45614,514.74544,903.16225
alzenau,736.32488,66.21271,0.00000,632.98106,261.08945,171.73195,466.33969,599.53713,604.29144,221.53011,...,109.61146,223.42977,182.60919,102.55432,1854.02875,738.13944,579.28138,164.60931,480.31750,1004.02994
amiens,1028.03563,667.40900,632.98106,0.00000,763.55469,471.23153,814.44144,559.88975,137.59623,520.23456,...,547.10169,742.15025,599.31831,701.89394,1658.59125,1145.22863,1081.74663,603.24275,346.00463,1411.11913
amt wachsenburg,821.21306,233.55459,261.08945,763.55469,0.00000,309.84056,540.47069,798.58956,817.31006,333.11663,...,308.66388,292.81534,381.66159,181.77248,2053.08125,487.07478,328.21669,329.34228,590.78588,752.96525


### c) log ostatniego uruchomienia przed crushem

In [ ]:
# import re
# import pandas as pd

# log_path = "./data/old/last_run.log"

# # Wczytanie całego pliku
# with open(log_path, "r", encoding="utf-8") as f:
#     log_text = f.read()

# # Wyrażenie regularne dopasowujące linie z wynikiem
# pattern = r"✅\s+([\wäöüÄÖÜßéèáàôîç\'\- ]+)\s+↔\s+([\wäöüÄÖÜßéèáàôîç\'\- ]+):\s+([\d\.]+)\s+km"

# # Dopasowanie wszystkich wystąpień
# matches = re.findall(pattern, log_text)

# # Utworzenie DataFrame
# df = pd.DataFrame(matches, columns=["city_a", "city_b", "distance_km"])

# # Konwersja kolumny distance_km do liczby
# df["distance_km"] = df["distance_km"].astype(float)

# print(f"✅ Odczytano {len(df)} wpisów z loga.")


✅ Odczytano 2608 wpisów z loga.


In [14]:
import re
import pandas as pd

with open("./data/old/last_run.log", "r", encoding="utf-8") as f:
    text = f.read()

pattern = r"✅\s+([^\↔:]+?)\s*↔\s*([^\↔:]+?):\s*([\d\.,]+)\s*[Kk][Mm]"

matches = re.findall(pattern, text)
df = pd.DataFrame(matches, columns=["city_a", "city_b", "distance_km"])

# Czyszczenie wartości
df["city_a"] = df["city_a"].str.strip()
df["city_b"] = df["city_b"].str.strip()
df["distance_km"] = df["distance_km"].str.replace(",", ".").astype(float)

print(f"✅ Odczytano {len(df)} wpisów")


✅ Odczytano 2795 wpisów


In [15]:
df

,city_a,city_b,distance_km
0,leupoldsgrün,ophasselt,752.7
1,leupoldsgrün,orsk,430.8
2,leupoldsgrün,osieczna,484.3
3,leupoldsgrün,osterweddingen,239.6
4,leupoldsgrün,pampow,484.2
...,...,...,...
2790,mannheim,vieux- thann,262.8
2791,mannheim,villefranche sur saone,595.3
2792,mannheim,villette-d'anthon,607.8
2793,mannheim,villmergen,327.1


In [16]:
updated = 0  # licznik zmian

for _, row in df.iterrows():
    a, b, dist = row["city_a"], row["city_b"], row["distance_km"]

    # sprawdź, czy oba miasta występują w macierzy
    if a in sample_cities_distance_de_matrix.index and b in sample_cities_distance_de_matrix.columns:
        if sample_cities_distance_de_matrix.at[a, b] == 0:
            # ustaw wartość dla (a,b) i (b,a)
            sample_cities_distance_de_matrix.at[a, b] = dist
            sample_cities_distance_de_matrix.at[b, a] = dist
            updated += 1

print(f"✅ Uzupełniono {updated} wartości w macierzy.")

✅ Uzupełniono 1070 wartości w macierzy.


In [15]:
sample_cities_distance_de_matrix

,agnadello,altfeld,alzenau,amiens,amt wachsenburg,andernach,andwil sg,ardon,aulnay sous bois,balve,...,worms,wörnitz,wörth am rhein,würzburg,xixona,zbaszynek,zgorzelec,zuzenhausen,zwijndrecht,łódź
city,,,,,,,,,,,,,,,,,,,,,
agnadello,0.00000,689.69838,736.32488,1028.03563,821.21306,807.32656,378.07547,451.18134,966.16325,900.30200,...,658.59669,579.96481,578.05044,657.16956,1506.68350,1285.57825,1088.16225,628.25413,1089.70238,1447.04850
altfeld,689.69838,0.00000,66.21271,667.40900,233.55459,206.15992,403.82131,624.65481,638.71938,278.81241,...,144.03942,160.91139,217.03716,40.03594,1879.14638,720.83656,523.42053,152.45614,514.74544,903.16225
alzenau,736.32488,66.21271,0.00000,632.98106,261.08945,171.73195,466.33969,599.53713,604.29144,221.53011,...,109.61146,223.42977,182.60919,102.55432,1854.02875,738.13944,579.28138,164.60931,480.31750,1004.02994
amiens,1028.03563,667.40900,632.98106,0.00000,763.55469,471.23153,814.44144,559.88975,137.59623,520.23456,...,547.10169,742.15025,599.31831,701.89394,1658.59125,1145.22863,1081.74663,603.24275,346.00463,1411.11913
amt wachsenburg,821.21306,233.55459,261.08945,763.55469,0.00000,309.84056,540.47069,798.58956,817.31006,333.11663,...,308.66388,292.81534,381.66159,181.77248,2053.08125,487.07478,328.21669,329.34228,590.78588,752.96525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zbaszynek,1285.57825,720.83656,738.13944,1145.22863,487.07478,788.49931,971.12050,1274.59625,1211.09188,674.51225,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
zgorzelec,1088.16225,523.42053,579.28138,1081.74663,328.21669,629.64119,773.70444,1072.69838,1136.57088,616.16569,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
zuzenhausen,628.25413,152.45614,164.60931,603.24275,329.34228,230.93961,336.50000,485.64628,566.04150,323.44791,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


### Zpis do pliku

In [14]:
path = './data/output/sample_de/sample_cities_distance_de_matrix.xlsx'

sample_cities_distance_de_matrix.to_excel(path, sheet_name="Cities_Distances_Matrix")